In [2]:
import pandas as pd
import numpy as np

from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer

from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder

In [24]:
pd.options.display.max_rows = 1000

In [25]:
df0 = pd.read_excel('train_data_github.xlsx')

In [26]:
# 고객별 구매 빈도 확인 위한 pivoting
df_pivot2 = df0.pivot_table(index = 'Cutomer_ID', values = 'ProductNo', aggfunc = 'count')

In [27]:
# IQR 기준 이상치 판단(Q3 + (Q3 - Q1)*1.5)
df_pivot2['ProductNo'].describe()

count    41121.000000
mean         2.431847
std          3.863633
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max        402.000000
Name: ProductNo, dtype: float64

In [28]:
# 2번 이상 구매 고객 sorting, 이상치 제거
member_index2 = df_pivot2[(df_pivot2['ProductNo'] > 1)&(df_pivot2['ProductNo'] < 5)].index
df = df0[df0['Cutomer_ID'].isin(member_index2)]

In [29]:
# add_info7, add_info8, and add_info9 에 대한 라벨 인코더 생성
encoder1 = LabelEncoder()
encoder2 = LabelEncoder()


# add_info7 열 인코딩
df.loc[:, 'CTG'] = encoder1.fit_transform(df['CTG'])

# add_info8 열 인코딩
df.loc[:, 'Sub_CTG'] = encoder2.fit_transform(df['Sub_CTG'])

# datetime data numeric으로 변환
df.loc[:, 'Order_Date'] = pd.to_numeric(df['Order_Date'])

C:\Users\J13868\AppData\Local\Temp\ipykernel_19280\53362519.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'CTG'] = encoder1.fit_transform(df['CTG'])
C:\Users\J13868\AppData\Local\Temp\ipykernel_19280\53362519.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'CTG'] = encoder1.fit_transform(df['CTG'])
C:\Users\J13868\AppData\Local\Temp\ipykernel_19280\53362519.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [30]:
# 각 고객의 구매 이력을 시계열 데이터로 변환
X = []
y = []
member_ids = []
maxlen = 0
for member_id in df['Cutomer_ID'].unique():
    member_df = df[df['Cutomer_ID'] == member_id].sort_values('Order_Date')
    for i in range(len(member_df) - 1):
        X.append(member_df.iloc[:i+1][['ProductNo', 'CTG', 'Sub_CTG' ]].values)
        y.append(member_df.iloc[i+1]['CTG'])
        member_ids.append(member_id)
        maxlen = max(maxlen, i+1)

# 시계열 데이터를 동일한 길이로 패딩
X = pad_sequences(X, maxlen=maxlen, dtype='float32')

# NaN 값을 대체
X[np.isnan(X)] = 0

In [43]:
# 데이터를 학습/테스트 세트로 분리
X_train, X_test, y_train, y_test, member_ids_train, member_ids_test = train_test_split(X, y, member_ids, test_size=0.2, random_state=0)

# LightGBM 모델 학습
model = LGBMClassifier(num_leaves=56,
                       min_data_in_leaf=30,
                       max_depth=7,
                       bagging_fraction=0.6,
                       feature_fraction=0.75,
                       random_state=0)
model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6


LGBMClassifier(bagging_fraction=0.6, feature_fraction=0.75, max_depth=7,
               min_data_in_leaf=30, num_leaves=56, random_state=0)

In [44]:
# 모델 평가
y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))

# 마지막 구매일에 구매한 제품들을 저장할 딕셔너리
last_purchase = {}
for member_id in df['Cutomer_ID'].unique():
    member_df = df[df['Cutomer_ID'] == member_id].sort_values('Order_Date')
    last_purchase_date = member_df.iloc[-1]['Order_Date']
    last_purchase_products = member_df[member_df['Order_Date'] == last_purchase_date]['CTG'].unique()
    last_purchase[member_id] = last_purchase_products

# 정답 여부를 저장할 리스트
correct = []
for i in range(len(y_pred)):
    if y_pred[i] in last_purchase[member_ids_test[i]]:
        correct.append(1)
    else:
        correct.append(0)

# 정확도 계산
accuracy = sum(correct) / len(correct)
print('Accuracy Score:', accuracy)

Accuracy Score: 0.8549869904596704


In [35]:
# 빈 데이터프레임 생성
results_df = pd.DataFrame(columns=['Cutomer_ID', 'Predicted_CTG'])

# 각 MemberID에 대하여 반복 수행
for member_id in df['Cutomer_ID'].unique():
    # Get the data for the current MemberID
    member_df = df[df['Cutomer_ID'] == member_id].sort_values('Order_Date')
    
    # Input Data 전처리
    X2 = member_df[['ProductNo','CTG', 'Sub_CTG']].values
    X2 = pad_sequences(X2[np.newaxis,:,:], maxlen=maxlen, dtype='float32')
    X2[np.isnan(X2)] = 0
    X2 = X2.reshape(1, -1)
    
    # 모델 예측
    y_pred2 = model.predict(X2)
    
    # 데이터 프레임에 결과값 입력
    results_df = pd.concat([results_df, pd.DataFrame({'Cutomer_ID': [member_id], 'Predicted_CTG': [y_pred2[0]]})], ignore_index=True)

In [36]:
# Summary 데이터 프레임 생성
Sector_df = pd.DataFrame(columns=['Cutomer_ID', 'A', 'B', 'C', 'D', 'E'])

# 각 MemberID에 대하여 반복작업 수행
for member_id in df['Cutomer_ID'].unique():
    # Get the purchase history for the current customer
    member_df2 = df[df['Cutomer_ID'] == member_id].sort_values('Order_Date')
    
    # 섹터별 구매 빈도 확인
    sector_counts = member_df2['Sector'].value_counts()
    
    # 데이터 프레임에 섹터별 열 생성
    Sector_df = pd.concat([Sector_df, pd.DataFrame({
        'Cutomer_ID': [member_id],
        'A': [sector_counts.get('A', 0)],
        'B': [sector_counts.get('B', 0)],
        'C': [sector_counts.get('C', 0)],
        'D': [sector_counts.get('D', 0)],
        'E': [sector_counts.get('E', 0)]
    })], ignore_index=True)

In [46]:
summary_df = pd.merge(Sector_df,results_df, on = 'Cutomer_ID', how = 'inner')

In [47]:
summary_df['Predicted_CTG'] = summary_df['Predicted_CTG'].astype(int)
summary_df['Predicted_CTG'] = encoder1.inverse_transform(summary_df['Predicted_CTG'])

In [48]:
summary_df

,Cutomer_ID,A,B,C,D,E,Predicted_CTG
0,325217,0,0,0,2,0,DA
1,734488,0,0,0,2,0,DA
2,666487,0,0,0,4,0,DA
3,265291,0,0,0,3,0,DA
4,219300,0,0,1,1,0,DA
...,...,...,...,...,...,...,...
18446,336560,0,0,0,2,0,DA
18447,896204,0,0,0,3,0,DA
18448,712827,0,0,0,4,0,DA
18449,855676,0,0,2,0,0,CA


In [41]:
# Bayes Search 통한 Hyper Parameter Fine Tuning

from skopt import BayesSearchCV
from skopt.space import Real, Integer

# 하이퍼 파라미터 검색 공간
param_grid = {
    'num_leaves': Integer(7, 63),
    'min_data_in_leaf': Integer(1, 30),
    'max_depth': Integer(-1, 7),
    'bagging_fraction': Real(0.6, 0.9),
    'feature_fraction': Real(0.6, 0.9)
}

# 베이지안 최적화
model = LGBMClassifier(random_state=0)
bayes_search = BayesSearchCV(model, param_grid, cv=5)
bayes_search.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 최적의 하이퍼 파라미터
best_params = bayes_search.best_params_
print('Best parameters:', best_params)

# LightGBM 모델 학습
model = LGBMClassifier(**best_params)
model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 모델 평가
y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))
print('Accuracy:', accuracy_score(y_test, y_pred))

[LightGBM] [Warning] feature_fraction is set=0.7205221970846976, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7205221970846976
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Warning] bagging_fraction is set=0.7071548401239501, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7071548401239501
[LightGBM] [Warning] feature_fraction is set=0.7205221970846976, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7205221970846976
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Warning] bagging_fraction is set=0.7071548401239501, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7071548401239501
[LightGBM] [Warning] feature_fraction is set=0.7205221970846976, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7205221970846976


[LightGBM] [Warning] feature_fraction is set=0.8724799788031286, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8724799788031286
[LightGBM] [Warning] min_data_in_leaf is set=9, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9
[LightGBM] [Warning] bagging_fraction is set=0.6528988524730549, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6528988524730549
[LightGBM] [Warning] feature_fraction is set=0.8724799788031286, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8724799788031286
[LightGBM] [Warning] min_data_in_leaf is set=9, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9
[LightGBM] [Warning] bagging_fraction is set=0.6528988524730549, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6528988524730549
[LightGBM] [Warning] feature_fraction is set=0.8724799788031286, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8724799788031286
[Lig

[LightGBM] [Warning] feature_fraction is set=0.6995509911404302, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6995509911404302
[LightGBM] [Warning] min_data_in_leaf is set=24, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=24
[LightGBM] [Warning] bagging_fraction is set=0.8940569209507632, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8940569209507632
[LightGBM] [Warning] feature_fraction is set=0.6995509911404302, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6995509911404302
[LightGBM] [Warning] min_data_in_leaf is set=24, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=24
[LightGBM] [Warning] bagging_fraction is set=0.8940569209507632, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8940569209507632
[LightGBM] [Warning] feature_fraction is set=0.6995509911404302, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6995509911404302


[LightGBM] [Warning] feature_fraction is set=0.6757686337539208, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6757686337539208
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] bagging_fraction is set=0.6520784767399761, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6520784767399761
[LightGBM] [Warning] feature_fraction is set=0.6757686337539208, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6757686337539208
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] bagging_fraction is set=0.6520784767399761, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6520784767399761
[LightGBM] [Warning] feature_fraction is set=0.6757686337539208, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6757686337539208


[LightGBM] [Warning] feature_fraction is set=0.8999996672762999, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8999996672762999
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.8407990813009354, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8407990813009354
[LightGBM] [Warning] feature_fraction is set=0.8999996672762999, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8999996672762999
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.8407990813009354, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8407990813009354
[LightGBM] [Warning] feature_fraction is set=0.8999996672762999, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8999996672762999
[Lig

...

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging

[LightGBM] [Warning] feature_fraction is set=0.7490807259171988, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7490807259171988
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.7740010250208583, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7740010250208583
[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=29
[LightGBM] [Warning] bagging_fraction is set=0.6273411534708451, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6273411534708451
[LightGBM] [Warning] feature_fraction is set=0.7740010250208583, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7740010250208583
[LightGBM] [Warning] min_data_

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=18, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=18
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=18, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=18
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=18, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=18
[LightGBM] [Warning] bagging

[LightGBM] [Warning] feature_fraction is set=0.8722819467884878, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8722819467884878
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] bagging_fraction is set=0.7198820130878475, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7198820130878475
[LightGBM] [Warning] feature_fraction is set=0.8722819467884878, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8722819467884878
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] bagging_fraction is set=0.7198820130878475, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7198820130878475
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=16, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=16
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=16, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=16
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=16, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=16
[LightGBM] [Warning] bagging

In [42]:
from sklearn.metrics import confusion_matrix

# Confusion Matrix 계산
cm = confusion_matrix(y_test, y_pred)

# Confusion Matrix DataFrame으로 변환
cm_df = pd.DataFrame(cm, columns=[f'Predicted {c}' for c in encoder1.classes_], index=[f'Actual {c}' for c in encoder1.classes_])

# 오류가 가장 많은 순으로 정렬
error_df = cm_df.stack().reset_index()
error_df.columns = ['Actual Class', 'Predicted Class', 'Count']
error_df = error_df[error_df['Actual Class'] != error_df['Predicted Class']].sort_values('Count', ascending=False)

# 문자열 삭제
error_df['Actual Class'] = error_df['Actual Class'].str.replace('Actual ', '')
error_df['Predicted Class'] = error_df['Predicted Class'].str.replace('Predicted ', '')


error_df

,Actual Class,Predicted Class,Count
45,DA,DA,3260
0,AA,AA,406
27,CB,CB,302
18,CA,CA,260
9,BA,BA,218
37,CC,DA,179
5,AA,DA,104
36,CC,CC,96
40,DA,AA,73
29,CB,DA,73
